d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Hyperopt Lab

The [Hyperopt library](https://github.com/hyperopt/hyperopt) allows for parallel hyperparameter tuning using either random search or Tree of Parzen Estimators (TPE). With MLflow, we can record the hyperparameters and corresponding metrics for each hyperparameter combination. You can read more on [SparkTrials w/ Hyperopt](https://github.com/hyperopt/hyperopt/blob/master/docs/templates/scaleout/spark.md).

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
- Learn how to distribute tuning tasks when training a single-node machine learning model by using `SparkTrials` class, rather than the default `Trials` class. 

> SparkTrials fits and evaluates each model on one Spark executor, allowing massive scale-out for tuning. To use SparkTrials with Hyperopt, simply pass the SparkTrials object to Hyperopt's fmin() function.

In [0]:
pip install hyperopt mlflow

Python interpreter will be restarted.
Requirement already satisfied: hyperopt in /local_disk0/.ephemeral_nfs/envs/pythonEnv-f889967b-3eb7-4568-9e7b-66c9eb789547/lib/python3.8/site-packages (0.2.5)
Collecting mlflow
 Downloading mlflow-1.18.0-py3-none-any.whl (14.2 MB)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from hyperopt) (1.15.0)
Requirement already satisfied: cloudpickle in /local_disk0/.ephemeral_nfs/envs/pythonEnv-f889967b-3eb7-4568-9e7b-66c9eb789547/lib/python3.8/site-packages (from hyperopt) (1.6.0)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (from hyperopt) (1.5.2)
Requirement already satisfied: tqdm in /local_disk0/.ephemeral_nfs/envs/pythonEnv-f889967b-3eb7-4568-9e7b-66c9eb789547/lib/python3.8/site-packages (from hyperopt) (4.61.2)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from hyperopt) (1.19.2)
Requirement already satisfied: networkx>=2.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-f889967b-3eb7-4568-9e7b-66c9eb789547/lib/python3.8/site-packages (from hyperopt) (2.5.1)
Requirement already satisfied: future in /local_disk0/.ephemeral_nfs/envs/pythonEnv-f889967b-3eb7-4568-9e7b-66c9eb789547/lib/python3.8/site-packages (from hyperopt) (0.18.2)
Collecting pyyaml>=5.1
 Downloading PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
Collecting prometheus-flask-exporter
 Downloading prometheus_flask_exporter-0.18.2.tar.gz (22 kB)
Collecting alembic<=1.4.1
 Downloading alembic-1.4.1.tar.gz (1.1 MB)
Collecting databricks-cli>=0.8.7
 Downloading databricks-cli-0.14.3.tar.gz (54 kB)
Requirement already satisfied: protobuf>=3.7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.0)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Collecting click>=7.0
 Downloading click-8.0.1-py3-none-any.whl (97 kB)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.24.0)
Collecting docker>=4.0.0
 Downloading docker-5.0.0-py2.py3-none-any.whl (146 kB)
Collecting querystring-parser
 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.1.5)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Collecting packaging
 Downloading packaging-21.0-py3-none-any.whl (40 kB)
Collecting Flask
 Downloading Flask-2.0.1-py3-none-any.whl (94 kB)
Collecting sqlalchemy
 Downloading SQLAlchemy-1.4.20-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
Collecting gitpython>=2.1.0
 Downloading GitPython-3.1.18-py3-none-any.whl (170 kB)
Collecting sqlparse>=0.3.1
 Downloading sqlparse-0.4.1-py3-none-any.whl (42 kB)
Collecting gunicorn; platform_system != "Windows"
 Downloading gunicorn-20.1.0-py3-none-any.whl (79 kB)
Requirement already satisfied: decorator<5,>=4.3 in /databricks/python3/lib/python3.8/site-packages (from networkx>=2.2->hyperopt) (4.4.2)
Collecting prometheus_client
 Downloading prometheus_client-0.11.0-py2.py3-none-any.whl (56 kB)
Collecting Mako
 Downloading Mako-1.1.4-py2.py3-none-any.whl (75 kB)
Collecting python-editor>=0.3
 Downloading python_editor-1.0.4-py3-none-any.whl (4.9 kB)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.8/site-packages (from alembic<=1.4.1->mlflow) (2.8.1)
Collecting tabulate>=0.7.7
 Downloading tabulate-0.8.9-py3-none-any.whl (25 kB)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (3.0.4)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (1.25.11)
Requirement already satisfied: idna<3,>=2.

In [0]:
%run "../Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Declared various utility methods: 
 Declared untilStreamIsReady( name:String ) to control workflow 
 
 All done!

Read in a cleaned version of the Airbnb dataset with just numeric features.

In [0]:
# did not work
# dbutils.fs.cp("dbfs:/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", "/", True)

Out[62]: True

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd

# does not work on DB community edition
# df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv").drop(["zipcode"], axis=1)

# workaround is to read using spark and then convert to pd dataframe
sparkDf = spark.read.option("header", "true").csv("dbfs:/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv").drop("zipcode")
df = sparkDf.select("*").toPandas()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1),
                                                    df[["price"]].values.ravel(),
                                                    test_size = 0.2,
                                                    random_state = 42)

Now we need to define an `objective_function` where you evaluate the [random forest's](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) predictions using R2.

In the code below, compute the `r2` and return it along with `STATUS_OK` (remember we are trying to maximize R2, so we need to return it as a negative value).

In [0]:
# TODO
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
from numpy import mean
from hyperopt import STATUS_OK
  
def objective_function(params):

  # set the hyperparameters that we want to tune:
  max_depth = params["max_depth"]
  max_features = params["max_features"]

  regressor = RandomForestRegressor(max_depth=max_depth, max_features=max_features, random_state=42)

  # Evaluate predictions
  r2 = mean(cross_val_score(regressor, X_train, y_train, cv=3))

  # Note: since we aim to maximize r2, we need to return it as a negative value ("loss": -metric)
  return {"loss": -r2, "status": STATUS_OK}

We need to define a search space for HyperOpt. Let the `max_depth` vary between 2-10, and `max_features` be one of: "auto", "sqrt", or "log2".

In [0]:
# TODO
from hyperopt import hp

search_space = {
  "max_depth": hp.randint("max_depth", 2, 10),
  "max_features": hp.choice("max_features", ["auto", "sqrt", "log2"])
}

Instead of using the default `Trials` class, you can leverage the `SparkTrials` class to trigger the distribution of tuning tasks across Spark executors. On Databricks, SparkTrials are automatically logged with MLflow.

`SparkTrials` takes 3 optional arguments, namely `parallelism`, `timeout`, and `spark_session`. You can refer to this [page](http://hyperopt.github.io/hyperopt/scaleout/spark/) to read more.

In the code below, fill in the `fmin` function.

In [0]:
# TODO
from hyperopt import fmin, tpe, STATUS_OK, SparkTrials
import mlflow

# Creating a parent run
with mlflow.start_run():
  # the number of models we want to evaluate
  num_evals = 8
  # set the number of models to be trained concurrently
#   spark_trials = SparkTrials(parallelism=2)
  best_hyperparam = fmin(fn=objective_function,
                         space=search_space,
                         algo=tpe.suggest,
#                          trials=spark_trials,
                         max_evals=num_evals)

  # get optimal hyperparameter values
  best_max_depth = best_hyperparam["max_depth"]
  best_max_features = best_hyperparam["max_features"]

  # train model on entire training data
  regressor = RandomForestRegressor(max_depth=best_max_depth, max_features=best_max_features, random_state=42)
  regressor.fit(X_train, y_train)

  # evaluate on holdout/test data
  r2 = regressor.score(X_test, y_test)
  
  # Log param and metric for the final model
  mlflow.log_param("max_depth", best_max_depth)
  mlflow.log_param("max_features", best_max_features)
  mlflow.log_metric("loss", r2)

0%| | 0/8 [00:00<?, ?trial/s, best loss=?] 12%|█▎ | 1/8 [00:01<00:07, 1.07s/trial, best loss: -0.6566746347967997] 25%|██▌ | 2/8 [00:03<00:11, 1.97s/trial, best loss: -0.6566746347967997] 38%|███▊ | 3/8 [00:04<00:07, 1.58s/trial, best loss: -0.6566746347967997] 50%|█████ | 4/8 [00:05<00:05, 1.40s/trial, best loss: -0.6608776555513131] 62%|██████▎ | 5/8 [00:07<00:03, 1.31s/trial, best loss: -0.6608776555513131] 75%|███████▌ | 6/8 [00:09<00:03, 1.65s/trial, best loss: -0.6608776555513131] 88%|████████▊ | 7/8 [00:11<00:01, 1.97s/trial, best loss: -0.6608776555513131]100%|██████████| 8/8 [00:13<00:00, 1.68s/trial, best loss: -0.6608776555513131]100%|██████████| 8/8 [00:13<00:00, 1.63s/trial, best loss: -0.6608776555513131]

Now you can compare all of the models using the MLflow UI. 

To understand the effect of tuning a hyperparameter:

0. Select the resulting runs and click Compare.
0. In the Scatter Plot, select a hyperparameter for the X-axis and loss for the Y-axis.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>